### Instalando las librerias necesarias

Para la correcta ejecucion del proyecto primero se deberan de instalar las librerias usadas para la implementacion del mismo.

In [45]:
!pip install anytree pandas
!pip install pytholog
!apt install swi-prolog
!pip install pykeen


The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.



### Implementacion del agente

In [46]:
import numpy as np
import pandas as pd
from anytree import NodeMixin, RenderTree
import pytholog as pl 
import math
import itertools

In [47]:
"""
Este metodo convierte cualquier lista de tareas sin orden en un arbol, para que el algoritmo de busqueda pueda luego seleccionar el mejor camino dentro del arbol. 
"""

# leer el archivo csv original
df = pd.read_csv("allTasks.csv")

# ordenar las tareas por su tiempo de inicio
df = df.sort_values(by=["TimeStart"])

# crear un diccionario para almacenar los padres de cada tarea
parents = {}

# recorrer cada tarea
for i, row in df.iterrows():
    # encontrar el tiempo de finalización de la tarea actual
    current_end_time = row["TimeStart"] + row["AvgTime"]
    # inicializar la distancia más cercana como infinito
    closest_distance = float("inf")
    # inicializar el padre más cercano como None
    closest_parent = None
    # buscar el padre más cercano en las tareas anteriores
    for j in range(i):
        # encontrar el tiempo de finalización de la tarea anterior
        previous_end_time = df.iloc[j]["TimeStart"] + df.iloc[j]["AvgTime"]
        # si la tarea anterior termina antes de que comience la tarea actual
        # y la distancia entre la tarea anterior y la actual es menor que la distancia más cercana actual
        # actualizar el padre más cercano y la distancia más cercana
        if previous_end_time <= row["TimeStart"] and (row["TimeStart"] - previous_end_time) < closest_distance:
            closest_distance = row["TimeStart"] - previous_end_time
            closest_parent = int(df.iloc[j]["TaskID"])
    # si se encontró un padre cercano, agregarlo al diccionario de padres
    if closest_parent is not None:
        parents[int(row["TaskID"])] = int(closest_parent)

# agregar la columna Parent al dataframe
df["Parent"] = df["TaskID"].map(parents)

# guardar el dataframe en un nuevo archivo csv
df.to_csv("taskTree.csv", index=False)


In [48]:
class Task:
    def __init__(self, id, taskType, name, avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves):
        self.id=id
        self.taskType=taskType
        self.name=name
        self.avgTime=avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
        self.onPath=False
    
    def __str__(self) -> str:
        """
        To String del objeto Task. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Task): Instancia de la clase Task.
        
        Returns:
            str: Representacion como texto del objeto Task.
        """
        return f"Name: {self.name}, AvgTime: {self.avgTime}, Urgency: {self.urgency}"


    """
    taskType -> 
      1- Alimentación​
      2- Aseo​
      3- Actividad de Mascota​
      4- Estudio​
      5- Salud Física​
      6- Plantas​
      7- Gustos​
      8- Actividad Social​
      9- Actividad de Autocuidado y Orden
    avgTime -> Tiempo promedio (minutos)
    timeStart -> Hora en que suele comenzar esta actividad
    deadline -> Dias para entregarla (max)
    urgency -> nivel de prioridad
    location -> lugar fisico
    isHobby -> Se sabe que sea un hobby
    recurrency -> Cuantos dias de la semana suele repetirse
    bestOn-> Mejor en mañana o en la tarde
    involves -> Sujetos terceros.
    """


class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self,taskType, name,  avgTime, timeStart, deadline, urgency, location, isHobby, recurrency, bestOn, involves, parent=None, children=None):
        super(Task, self).__init__()
        self.taskType=taskType
        self.name = name
        self.avgTime = avgTime
        self.timeStart=timeStart
        self.deadline=deadline
        self.location=location
        self.urgency=urgency
        self.isHobby=isHobby
        self.recurrency=recurrency
        self.bestOn=bestOn
        self.involves=involves
        self.parent = parent
        self.onPath=False
        if children:
            self.children = children

In [49]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency



    def __str__(self):
        """
        To String del objeto Choice. 
        Llevar los atributos del objeto a una forma legible a la
        hora de imprimir el objeto.

        Args:
            self (Choice): Instancia de la clase Choice.
        
        Returns:
            str: Representacion como texto del objeto Choice.
        """
        return self.move + ": " + str(self.urgency)
    

In [50]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características


class ScheduleAgent:
  def __init__(self, name:str, age:int, gender:str, taskList:pd.DataFrame, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.taskList = taskList
    self.notification = comments
    self.comments = comments
    self.rootNode = self.buildTree() #creo el arbol al inicializar el objeto
    
    
  
  def getNodesList(self) -> list:
    """
    Crear una lista de nodos del arbol de tareas de acuerdo a las
    tareas almacenadas en un archivo CSV.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        list: Lista de nodos del arbol de tareas, cada nodo representa una tarea.
    """

# TaskID,TaskType,Name,AvgTime,TimeStart,Deadline,Urgency,location,isHobby,recurrency,bestOn,involves,Parent,Children

    nodeList = []
    for i in range(0, len(self.taskList["Name"])):
        nodeList.append(TaskClass(
            taskType = self.taskList["TaskType"][i],
            name = self.taskList["Name"][i],
            avgTime = self.taskList["AvgTime"][i],
            timeStart = self.taskList["TimeStart"][i],
            deadline = self.taskList["Deadline"][i],
            urgency = self.taskList["Urgency"][i],
            location = self.taskList["Location"][i],
            isHobby = self.taskList["IsHobby"][i],
            recurrency = self.taskList["Recurrency"][i],
            bestOn = self.taskList["BestOn"][i],
            involves = self.taskList["Involves"][i],
        ))
    return nodeList
  
  
  
  def setParentNodes(self, nodeList:list) -> TaskClass:
    """
    Colocar a cada uno de los nodos del arbol de tareas su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        nodeList (list): Lista de nodos del arbol de tareas, cada nodo representa una tarea.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(self.taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo

  
  
  def buildTree(self) -> TaskClass: 
    """
    Armar el arbol de tareas.
    En primera instancia crea una lista de nodos de tareas para 
    despues asignarles su respectivo nodo padre.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.

    Returns:
        TaskClass: Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
    """
    nodeList = self.getNodesList()
    rootNode = self.setParentNodes(nodeList)
    return rootNode #retorno root del arbol ya que desde root lo puedo recorrer todo
    
    
    
  def printTaskTree(self) -> None:
    """
    Imprimir por completo el arbol de tareas. 
    
    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
    """
    for pre, fill, node in RenderTree(self.rootNode):
        print("%s%s" % (pre, node.name))   
  
  
  
  def miniMax(self, rootNode:Task, isMax:bool) -> Choice:
    """
    Algoritmo de busqueda miniMax para determinar la mayor cantidad de tareas de la mayor urgencia posible
    a completar en el minimo tiempo posible.

    Args:
        self (ScheduleAgent): Instancia de la clase ScheduleAgent.
        rootNode (Task): Nodo raiz del arbol de tareas, desde el nodo raiz se puede recorrer la totalidad del arbol.
        isMax (bool): ?

    Returns:
        Choice: Eleccion de rama y nodo del algortimo miniMax
    """
    children = rootNode.children
    try:
        l_choice = self.miniMax(children[0], not isMax)
        r_choice = self.miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", self.rootNode.urgency)

In [51]:
horario = {}

def agregar_tarea(node):
    horario[node.name] = node

def ver_horario():
    print(horario)

In [52]:
#inicializando el agente para el usuario Ricardo
myAgent = ScheduleAgent(name="Ricardo",
                        age=36,
                        gender="Male",
                        taskList=pd.read_csv('taskList.csv'),
                        comments = "")

isMax = True
currentNode = myAgent.rootNode
agregar_tarea(currentNode)

print("Comenzamos nuestro recorrido en el nodo: "+str(currentNode.name)+", con una urgencia de: "+str(currentNode.urgency))
currentNode.onPath=True

while (True):
    # run minimax on current node
    agentChoice = myAgent.miniMax(currentNode, isMax) 
    
    # make choice based on minimax search
    if (agentChoice.move == "left"):
        print ("Nos movemos a la izquierda con una urgencia de: " + str(currentNode.children[0].urgency)+ ", a la tarea: "+str(currentNode.children[0].name))
        currentNode.children[0].onPath=True
        currentNode = currentNode.children[0]
        agregar_tarea(currentNode)
    elif (agentChoice.move == "right"):
        print ("Nos movemos a la derecha con una urgencia de: " + str(currentNode.children[1].urgency)+ ", a la tarea: "+str(currentNode.children[1].name))
        currentNode.children[1].onPath=True
        currentNode = currentNode.children[1]
        agregar_tarea(currentNode)
    elif (agentChoice.move == "end"):
        print ("Hemos llegado al final del mejor recorrido")
        break

Comenzamos nuestro recorrido en el nodo: desayunar, con una urgencia de: 5
Nos movemos a la derecha con una urgencia de: 5, a la tarea: sacar al perro
Nos movemos a la derecha con una urgencia de: 7, a la tarea: regar plantas
Nos movemos a la derecha con una urgencia de: 6, a la tarea: jugar con el perro
Hemos llegado al final del mejor recorrido


In [53]:
ver_horario()

{'desayunar': <__main__.TaskClass object at 0x125fd2290>, 'sacar al perro': <__main__.TaskClass object at 0x125fd2230>, 'regar plantas': <__main__.TaskClass object at 0x125fd2ec0>, 'jugar con el perro': <__main__.TaskClass object at 0x125fd3e50>}


In [54]:
# PARA ORGANIZAR LAS TAREAS SEGUN SUS ATRIBUTOS.

dictTasks = {}
dictHobbies = {}
dictHabits = {}

In [55]:
# AÑADIR LAS TAREAS DE LA ESTRUCTURA ANYTREE AL ARCHIVO DE LOGICA DE PROLOG


# reinicia entre corridas el txt, aqui se ponen la reglas basicas de la logica del agente
with open("logica_agente.txt", "w") as archivo:
    archivo.write("sameLocation(X,Y) :- inLocation(X,Z), inLocation(Y,Z) -> sameLocation(X,Y). ")
archivo.close()


# METODO PARA AÑADIR LOS TASKS
def addAllTasks(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                    dictTasks[node.name] = node
                    archivo.write("isTask("+node.name + ")" + '\n')
                    archivo.write("inLocation("+node.name + "," + node.location +")" + '\n')
                    archivo.write("hasDeadline("+node.name + "," + str(node.deadline) +")" + '\n')
                    archivo.write("isBestOn("+node.name + "," + node.bestOn +")" + '\n')
                    if node.onPath == True:
                        archivo.write("inTodaysSchedule("+node.name + ")" + '\n')
                    if node.involves != "" and node.involves != np.nan:
                        archivo.write("involvesWho("+node.name + "," + str(node.involves) +")" + '\n')
                    
    archivo.close()

# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HOBBIES
def addHobbies(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.isHobby == True: 
                    dictHobbies[node.name] = node
                    archivo.write("isHobby("+node.name + ")" + '\n')
    archivo.close()


# METODO PARA FILTRAR LAS ACTIVIDADES QUE SON HABITOS
def addHabits(node):
    with open('logica_agente.txt', mode='a') as archivo:
        for node in node.descendants:
            if isinstance(node, Task):
                if node.recurrency >= 5: 
                    dictHabits[node.name] = node
                    archivo.write("isHabit("+node.name + ")" + '\n')
    archivo.close()
                
               

addAllTasks(myAgent.rootNode)
addHobbies(myAgent.rootNode)
addHabits(myAgent.rootNode)


print("Tasks: " + str(dictTasks))
print("Hobbies: " + str(dictHobbies))
print("Habits: " + str(dictHabits))


# Base de conocimiento

KB2 = pl.KnowledgeBase("KB2")
KB2.from_file("logica_agente.txt")



Tasks: {'bañarse': <__main__.TaskClass object at 0x125fd1cf0>, 'hacer la tarea': <__main__.TaskClass object at 0x125fd1a50>, 'almorzar': <__main__.TaskClass object at 0x125fd3880>, 'hacer ejercicio': <__main__.TaskClass object at 0x125fd3850>, 'estudiar para el parcial': <__main__.TaskClass object at 0x125fd2200>, 'escuchar musica': <__main__.TaskClass object at 0x125fd3820>, 'leer documentacion': <__main__.TaskClass object at 0x125fd37f0>, 'sacar al perro': <__main__.TaskClass object at 0x125fd2230>, 'hacer estiramientos': <__main__.TaskClass object at 0x125fd2e90>, 'ver netflix': <__main__.TaskClass object at 0x125fd3c40>, 'reunirse con amigos': <__main__.TaskClass object at 0x125fd3e20>, 'regar plantas': <__main__.TaskClass object at 0x125fd2ec0>, 'ordenar el cuarto': <__main__.TaskClass object at 0x125fd3fa0>, 'jugar con el perro': <__main__.TaskClass object at 0x125fd3e50>}
Hobbies: {'escuchar musica': <__main__.TaskClass object at 0x125fd3820>, 'ver netflix': <__main__.TaskClass 

In [56]:
# Ejemplos de Queries

# Queries sobre nuestros datos

print("Preguntamos sobre una tarea que no esta en el dataframe")
print(KB2.query(pl.Expr("isTask(desayunar)")))

print("Preguntamos datos sobre una tarea que sí esta en el dataframe")
print(KB2.query(pl.Expr("isTask(almorzar)")))

print("Preguntamos por clasificaciones")
print(KB2.query(pl.Expr("isHobby(almorzar)")))
print(KB2.query(pl.Expr("isHabit(almorzar)")))

print("Preguntamos datos sobre relaciones entre tareas")
print(KB2.query(pl.Expr("sameLocation(almorzar,regar plantas)")))

print("Distinguimos que tareas son las que se agregaron al horario de hoy ")
print(KB2.query(pl.Expr("inTodaysSchedule(almorzar)"))) 
print(KB2.query(pl.Expr("inTodaysSchedule(regar plantas)")))

Preguntamos sobre una tarea que no esta en el dataframe
['No']
Preguntamos datos sobre una tarea que sí esta en el dataframe
['Yes']
Preguntamos por clasificaciones
['No']
['Yes']
Preguntamos datos sobre relaciones entre tareas
['No']
Distinguimos que tareas son las que se agregaron al horario de hoy 
['No']
['Yes']
